In [1]:
import pandas as pd 
from pandas import Series , DataFrame
import numpy as np 
import scipy as sp
import ipywidgets as widgets
from ipywidgets import interactive
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import *
from bokeh.plotting import output_file,figure
from bokeh.application.handlers import FunctionHandler
from bokeh.models import Select


In [2]:
import glob
from IPython.core.pylabtools import figsize


In [3]:
path =r'Datasets/'
allFiles = glob.glob(path + "/*.csv")
dfs = pd.concat([pd.read_csv(f) for f in (allFiles)],ignore_index=True)

In [4]:
dfs['Date'] = pd.to_datetime(dfs['Date'])

In [5]:
dfs['day_of_week'] = dfs.Date.dt.dayofweek

In [6]:
from scipy.stats import gaussian_kde

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
station_list  = list(dfs['Station Name.1'].astype('category').cat.categories)

In [9]:
from bokeh.application import Application


In [10]:
dims_value = list(dfs.columns[9:14])

In [11]:
from bokeh.models import Slider

# Histogram

In [12]:
def modify_doc(doc):
    def create_figure():
        current_name = station_name.value
        current_dim = dims_name.value
        data = dfs[dfs['Station Name.1']==current_name]
        x = np.linspace(min(data[current_dim]),max(data[current_dim]),freq_slider.value*10)
        pdf = gaussian_kde(data[current_dim])
        p = figure(plot_height = 500,title=current_name)
        p.line(x,pdf(x))
        hist, edges = np.histogram(data[current_dim],density=True, bins=freq_slider.value)
        p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)
        return p

    def update_station(attr, old, new):
        layout.children[1] = create_figure()
    
    def update_dims(attr, old, new):
        layout.children[1] = create_figure()
        
    def update_bins(attr, old, new):
        layout.children[1] = create_figure()
    
    # Controls
    station_name = Select(title="Station Name:", options=station_list, value=station_list[0])
    station_name.on_change('value', update_station)
    
    dims_name = Select(title="Dimensions:", options=dims_value, value=dims_value[0])
    dims_name.on_change('value', update_dims)
    
    freq_slider = Slider(start=1, end=20, value=5, step=1,
                     title="bins") 
    
    freq_slider.on_change('value',update_bins)
    
    controls = widgetbox([station_name,dims_name,freq_slider], width=200)
    p = create_figure()
    layout = row(controls, p)
    doc.add_root(layout)

handler = FunctionHandler(modify_doc)
app = Application(handler)

In [13]:
doc = app.create_document()

In [14]:
show(app,notebook_url="localhost:8888")

# Time series 

In [15]:
from bokeh.models.formatters import DatetimeTickFormatter

In [17]:
def modify_doc_time(doc):
    def create_time_figure():
        current_name = station_name.value
        current_dim = dims_name.value
        
        data = dfs[dfs['Station Name.1']==current_name].sort_values(['Date'])
        
        data['Date']=data['Date'].dt.date
        p = figure(x_axis_type='datetime',plot_height = 500,plot_width=1000,title=current_name,y_axis_type="linear")
        
        p.line(data['Date'],data[current_dim])
        return p

    def update_station(attr, old, new):
        layout.children[1] = create_time_figure()
    
    def update_dims(attr, old, new):
        layout.children[1] = create_time_figure()
        
 
    # Controls
    station_name = Select(title="Station Name:", options=station_list, value=station_list[0])
    station_name.on_change('value', update_station)
    
    dims_name = Select(title="Dimensions:", options=dims_value, value=dims_value[0])
    dims_name.on_change('value', update_dims)
    
    controls = widgetbox([station_name,dims_name], width=200)
    p = create_time_figure()
    layout = row(controls, p)
    doc.add_root(layout)

handler_ts = FunctionHandler(modify_doc_time)
app_ts = Application(handler_ts)

In [18]:
doc_ts = app_ts.create_document()

In [20]:
show(app_ts,notebook_url="localhost:8888")